# PydanticAI fundamentals

In [1]:
from pydantic_ai import Agent

agent = Agent(model = "google-gla:gemini-2.5-flash")

result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result

AgentRunResult(output='**Name:** Linnea Karlsson\n**Role:** Software Developer\n**Location:** Stockholm, Sweden\n**Focus:** Backend systems (Java, Spring Boot), cloud infrastructure (AWS).')

In [3]:
print(result.output)

**Name:** Linnea Karlsson
**Role:** Software Developer
**Location:** Stockholm, Sweden
**Focus:** Backend systems (Java, Spring Boot), cloud infrastructure (AWS).


## use pydantic to structure output

In [6]:
from pydantic import BaseModel, Field

class Employee(BaseModel):
    name: str = Field(description="first name and last name of the person")
    age: int = Field(gt=17, lt=67, description="age of a person")
    salary: int = Field(gt=30_000, lt=70_000, description="salary of the employee, the more senior the role, the more salary")
    role: str = Field(description="always include seniority level in the role (junior, senior, expert)")
    description: str = Field(description="short description (max 2 sentences) of the role, include keywords")

result = await agent.run("Give me an IT employee working as Data Engineer in Sweden", output_type=Employee)

result

AgentRunResult(output=Employee(name='Erik Svensson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.'))

In [7]:
employee = result.output
employee

Employee(name='Erik Svensson', age=35, salary=65000, role='Senior Data Engineer', description='Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.')

In [8]:
employee.description, employee.name

('Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.',
 'Erik Svensson')

In [9]:
isinstance(employee, BaseModel)

True

In [11]:
employee.model_dump()

{'name': 'Erik Svensson',
 'age': 35,
 'salary': 65000,
 'role': 'Senior Data Engineer',
 'description': 'Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets.'}

In [12]:
employee.model_dump_json()

'{"name":"Erik Svensson","age":35,"salary":65000,"role":"Senior Data Engineer","description":"Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets."}'

In [13]:
print(employee.model_dump_json(indent=3))

{
   "name": "Erik Svensson",
   "age": 35,
   "salary": 65000,
   "role": "Senior Data Engineer",
   "description": "Designs, builds, and maintains scalable data pipelines and ETL processes. Focuses on optimizing data flow and architecture for large datasets."
}
